In [1]:
#include <algorithm>
#include <bitset>
#include <iostream>
#include <sstream>
#include <utility>

#include <xcpp/xdisplay.hpp>

#include <CArrayDefs/CArrayDefs.h>

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    xeus::xjson mime_bundle_repr(const html& a)
    {
        auto bundle = xeus::xjson::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

// Dump contents of 
template <typename T, typename V>
void dump_array(V a, const char *separator=" ", std::ostream &out=std::cout) {
    std::transform(a.data, a.data + a.length,
                   std::ostream_iterator<T>(out, separator), [](decltype(*a.data) v) {
                     return T(v);
                   });
}

template <typename T, typename V>
std::string concat_array(V a, const char *separator=" ") {
    std::stringstream output;
    dump_array<T>(a, separator, output);
    return output.str();
}

template <typename T, typename V>
auto display_array(V a, const char *separator=" ") {
    return ht::html(R"(
    <div>
    <style scoped="">
        .dataframe tbody tr th:only-of-type {
            vertical-align: middle;
        }

        .dataframe tbody tr th {
            vertical-align: top;
        }

        .dataframe thead th {
            text-align: right;
        }
    </style>
    <table border="1" class="dataframe">
      <tbody>
        <tr>
          <td>
             )" + concat_array<T>(a, "</td></tr><tr><td>") +
             R"(
          </td>
        </tr>
      </tbody>
    </table>
    </div>)");
}


struct Switch {
  uint8_t board;
  uint8_t port;
  uint8_t bit;
};

In [8]:
inline Switch channel_to_switch(uint8_t channel) {
    /*
     *  - 5 IO register ports per switching board.
     *  - 8 bits per IO register port.
     *  - Channels in LSB-first order within each IO register port.
     */
    const auto ports_per_board = 5;
    const auto channels_per_port = 8;
    const auto channels_per_board = ports_per_board * channels_per_port;
    
    Switch _switch;
    _switch.board = channel / channels_per_board;
    _switch.port = (channel % channels_per_board) / channels_per_port;
    _switch.bit = channel % channels_per_port;
    return _switch;
}

input_line_15:1:15: error: inline declaration of 'channel_to_switch' follows non-inline definition
inline Switch channel_to_switch(uint8_t channel) {
              ^
input_line_9:1:8: note: previous definition is here
Switch channel_to_switch(uint8_t channel) {
       ^


Interpreter Error: 

In [3]:
uint8_t switch_to_channel(Switch const &_switch) {
  /*
   *  - 5 IO register ports per switching board.
   *  - 8 bits per IO register port.
   *  - Channels in LSB-first order within each IO register port.
   */

  // Port number within all IO register ports concatenated.
  const auto ports_per_board = 5;
  const auto channels_per_port = 8;
  const auto channels_per_board = ports_per_board * channels_per_port;
    
  return _switch.board * channels_per_board + _switch.port * channels_per_port + _switch.bit;
}

In [4]:
void pack_channels(UInt8Array const &channels, UInt8Array &packed_channels) {
  const auto ports_per_board = 5;
  packed_channels.length = 0;
    
  for (auto i = 0; i < channels.length; i++) {
    const Switch _switch = channel_to_switch(channels.data[i]);
    const uint8_t byte_i = _switch.board * ports_per_board + _switch.port;
      
    if (byte_i + 1 >= packed_channels.length) {
        for (auto board_i = packed_channels.length; board_i < _switch.board + 1; board_i++) {
            for (auto port_j = 0; port_j < ports_per_board + 1; port_j++) {
                packed_channels.data[_switch.board * ports_per_board + port_j] = 0;
            }
        }
        packed_channels.length = (_switch.board + 1) * ports_per_board;
    }
      
    packed_channels.data[byte_i] |= 1 << _switch.bit;
  }
}

In [5]:
void unpack_channels1(UInt8Array const &packed_channels, UInt8Array &channels) {
  const auto ports_per_board = 5;
  const auto channels_per_port = 8;
  const auto channels_per_board = ports_per_board * channels_per_port;
    
  channels.length = 0;
    
  for (auto i = 0; i < packed_channels.length; i++) {
    uint8_t board_i = i / ports_per_board;
    uint8_t port_i = i % ports_per_board;
      
    for (uint8_t j = 0; j < 8; j++) {
        if (packed_channels.data[i] & (1 << j)) {
            const Switch _switch = {board_i, port_i, j};
            uint8_t channel_ij = switch_to_channel(_switch);
            std::cout << static_cast<int>(_switch.board) << ", "
                      << static_cast<int>(_switch.port) << ", " 
                      << static_cast<int>(_switch.bit) << ", " 
                      << static_cast<int>(channel_ij)
                      << std::endl;
            channels.data[channels.length++] = channel_ij;
        }
    }
  }
}

In [6]:
uint8_t packed_channels_buffer[15];
uint8_t unpacked_channels_buffer[120];
UInt8Array packed_channels = UInt8Array_init(0, packed_channels_buffer);
UInt8Array unpacked_channels = UInt8Array_init(0, unpacked_channels_buffer);

uint8_t channels[] = {1, 5, 38, 119};

In [7]:
pack_channels(UInt8Array_init(sizeof(channels), channels), packed_channels);
unpack_channels1(packed_channels, unpacked_channels);

xcpp::display(display_array<int>(UInt8Array_init(sizeof(channels) / sizeof(channels[0]), channels)));
xcpp::display(display_array<std::bitset<8>>(packed_channels));
xcpp::display(display_array<int>(unpacked_channels));

for (auto i = 0; i < unpacked_channels.length; i++) {
    if (channels[i] != unpacked_channels.data[i]) {
        std::cerr << "error: element " << i << " mismatch: "
                  << static_cast<int>(channels[i]) << " != "
                  << static_cast<int>(unpacked_channels.data[i]) << std::endl;
        break;
    }
}

0, 0, 1, 1
0, 0, 5, 5
0, 4, 6, 38
2, 4, 7, 119


1
5
38
119
""


00100010
00000000
00000000
00000000
01000000
00000000
00000000
00000000
00000000
00000000
00000000


1
5
38
119
""
